In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pickle as pk
import os
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [12]:
!pwd

/home/akshara/coding/Phenotyping/preprocess


In [17]:
data_dir = "./mimic-iii"
files_dir = "../dump_files"

In [17]:
pheno = pd.read_csv(os.path.join(files_dir, 'PHENOTYPE.csv'))

In [18]:
pheno

,HADM_ID,SUBJECT_ID,ROW_ID,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,BATCH.ID,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEMENTIA,DEPRESSION,DEVELOPMENTAL.DELAY.RETARDATION,NON.ADHERENCE,NONE,OBESITY,OPERATOR,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS,UNSURE
0,100103,3365,17170,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JTW,0,0,0
1,100103,3365,17170,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,ETM,0,0,0
2,100137,27290,50828,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JF,0,0,0
3,100137,27290,50828,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JW,0,0,0
4,100473,5525,52670,0,0,0,0,18Feb16,0,0,0,0,0,0,1,0,JW,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,199807,23549,2787,0,1,1,0,07Dec15,0,0,0,0,0,0,0,0,ETM,0,0,0
2272,199883,7029,18731,0,0,0,1,18Feb16,0,0,0,0,0,1,0,0,JW,0,0,0
2273,199883,7029,18731,0,0,0,1,18Feb16,0,0,0,0,0,1,0,0,JF,0,0,0
2274,199884,1931,8180,0,1,0,0,16Mar16,0,0,0,1,0,0,0,0,DAG,0,0,0


In [19]:
pheno  = pheno.drop(['ROW_ID', 'BATCH.ID', 'SUBJECT_ID', 'OPERATOR', 'UNSURE', 'DEVELOPMENTAL.DELAY.RETARDATION',
                   'NON.ADHERENCE', 'DEMENTIA', 'NONE'], axis=1)

In [21]:
pheno.columns

Index(['HADM_ID', 'ADVANCED.CANCER', 'ADVANCED.HEART.DISEASE',
       'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'CHRONIC.PAIN.FIBROMYALGIA',
       'DEPRESSION', 'OBESITY', 'OTHER.SUBSTANCE.ABUSE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS'],
      dtype='object')

In [22]:
pheno = pheno.groupby('HADM_ID').max().reset_index()

In [23]:
notes = pd.read_csv(os.path.join(files_dir, 'notes_rad.csv'))

In [24]:
df = notes.merge(pheno, on='HADM_ID')

In [55]:
df1 = notes1.merge(df, on='HADM_ID')

In [25]:
df

,HADM_ID,TEXT,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEPRESSION,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
0,100103.0,pm chest pa lat clip reason f u admitting diag...,0,0,0,0,0,0,0,0,0,0
1,100473.0,chest portable ap clip reason please assess in...,0,0,0,0,0,0,0,1,0,0
2,100548.0,pm hip unilat min views left clip reason pleas...,0,0,0,0,1,0,0,0,0,0
3,100654.0,chest portable ap clip reason eval pna effusio...,0,0,0,1,0,0,0,0,0,0
4,100960.0,pm chest pa lat clip reason infiltrate medical...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
636,199365.0,pm ct head w contrast clip reason ms changes h...,0,1,0,0,0,0,1,0,0,0
637,199576.0,pm chest portable ap clip reason r pneumothora...,0,0,0,0,0,0,0,0,0,0
638,199652.0,ercp biliary pancreas gi unit clip reason bile...,1,0,0,0,0,0,1,0,0,0
639,199807.0,pm chest portable ap physician reason eval pla...,0,1,1,0,0,0,0,0,0,0


In [15]:
pheno.columns

Index(['HADM_ID', 'ADVANCED.CANCER', 'ADVANCED.HEART.DISEASE',
       'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'CHRONIC.PAIN.FIBROMYALGIA',
       'DEPRESSION', 'OBESITY', 'OTHER.SUBSTANCE.ABUSE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS'],
      dtype='object')

In [37]:
notes1 = pd.read_csv(os.path.join(files_dir, 'notes_rad.csv'))

In [40]:
text1 = notes1.TEXT.tolist()
text = notes.TEXT.tolist()

In [41]:
s2 = " ".join(text)
s1 = " ".join(text1)
s = s1+ " " + s2
with open("./disch_rad_notes.txt", 'w') as f:
    f.write(s)

In [5]:
len(notes['NOTES'].min())

37991

In [19]:
field_length = notes.TEXT.astype(str).map(len)
#print(notes.loc[field_length.argmax(), 'NOTES'])
sm = notes.loc[field_length.argmin(), 'TEXT']
lg = notes.loc[field_length.argmax(), 'TEXT']

print(len(sm),len(lg))

153 51779


In [21]:
text = notes['TEXT'].tolist()
avg = sum(map(len, text))/float(len(text))

In [22]:
avg

6790.64892083602

In [19]:
notes1

,HADM_ID,TEXT
0,100001.0,chest pa lat clip reason eval infiltrate medic...
1,100003.0,pm liver gallbladder us single organ clip reas...
2,100006.0,chest portable ap clip reason please assess in...
3,100009.0,pm chest pa lat clip reason pre op admitting d...
4,100010.0,pm chest portable ap different physician reaso...
...,...,...
45521,199993.0,chest portable ap clip reason r effusion medic...
45522,199994.0,pm chest portable ap clip reason pt self extub...
45523,199995.0,pm abdomen u complete study duplex dopp abd pe...
45524,199998.0,pm chest port line placement clip reason line ...


In [20]:
df = notes.merge(pheno, on='HADM_ID')

In [39]:
df = df[['HADM_ID', 'TEXT','ADVANCED.CANCER', 'ADVANCED.HEART.DISEASE',
       'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'CHRONIC.PAIN.FIBROMYALGIA',
       'DEPRESSION', 'OBESITY', 'OTHER.SUBSTANCE.ABUSE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS']]

In [21]:
df

,HADM_ID,TEXT,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEPRESSION,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
0,100103.0,admission date discharge date date birth sex f...,0,0,0,0,0,0,0,0,0,0
1,100137.0,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,0,0,0
2,100473.0,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,1,0,0
3,100485.0,admission date discharge date date birth sex f...,0,0,0,0,0,0,1,0,0,1
4,100548.0,admission date discharge date date birth sex s...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
808,199576.0,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,0,0,0
809,199652.0,admission date discharge date date birth sex f...,1,0,0,0,0,0,1,0,0,0
810,199807.0,admission date discharge date date birth sex f...,0,1,1,0,0,0,0,0,0,0
811,199883.0,admission date discharge date date birth sex s...,0,0,0,1,0,0,0,0,0,0


In [22]:
df1 = notes1.merge(pheno, on='HADM_ID')

In [20]:
notes1 = pd.read_csv(os.path.join(files_dir, 'nurse_phys.csv'))

In [22]:
df1 = df1[['HADM_ID', 'NOTES', 'ADVANCED.CANCER', 'ADVANCED.HEART.DISEASE',
       'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'CHRONIC.PAIN.FIBROMYALGIA',
       'DEMENTIA', 'DEPRESSION', 'DEVELOPMENTAL.DELAY.RETARDATION',
       'NON.ADHERENCE', 'NONE', 'OBESITY', 'OTHER.SUBSTANCE.ABUSE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS']]

In [23]:
df1

,HADM_ID,TEXT,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEPRESSION,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
0,100103.0,pm chest pa lat clip reason f u admitting diag...,0,0,0,0,0,0,0,0,0,0
1,100473.0,chest portable ap clip reason please assess in...,0,0,0,0,0,0,0,1,0,0
2,100548.0,pm hip unilat min views left clip reason pleas...,0,0,0,0,1,0,0,0,0,0
3,100654.0,chest portable ap clip reason eval pna effusio...,0,0,0,1,0,0,0,0,0,0
4,100960.0,pm chest pa lat clip reason infiltrate medical...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
636,199365.0,pm ct head w contrast clip reason ms changes h...,0,1,0,0,0,0,1,0,0,0
637,199576.0,pm chest portable ap clip reason r pneumothora...,0,0,0,0,0,0,0,0,0,0
638,199652.0,ercp biliary pancreas gi unit clip reason bile...,1,0,0,0,0,0,1,0,0,0
639,199807.0,pm chest portable ap physician reason eval pla...,0,1,1,0,0,0,0,0,0,0


In [18]:
df1.to_csv(os.path.join(files_dir, 'pheno_icd.csv'), index=False)

In [3]:
df1 = pd.read_csv(os.path.join(files_dir, 'pheno_disch_rad.csv'))

In [6]:
df1.sum(axis=0)

HADM_ID                                                                                9.63329e+07
TEXT_x                                           pm chest pa lat clip reason f u admitting diag...
TEXT_y                                           admission date discharge date date birth sex f...
ADVANCED.CANCER                                                                                 59
ADVANCED.HEART.DISEASE                                                                         145
ADVANCED.LUNG.DISEASE                                                                           83
ALCOHOL.ABUSE                                                                                   90
CHRONIC.NEUROLOGICAL.DYSTROPHIES                                                               153
CHRONIC.PAIN.FIBROMYALGIA                                                                      126
DEPRESSION                                                                                     195
OBESITY   

In [25]:
df1.tail(2)

,HADM_ID,TEXT,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEPRESSION,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
639,199807.0,pm chest portable ap physician reason eval pla...,0,1,1,0,0,0,0,0,0,0
640,199883.0,pm chest pa lat clip reason pneumonia admittin...,0,0,0,1,0,0,0,0,0,0


In [26]:
dd = df.tail(2)['TEXT'].tolist()
dd[0]

'admission date discharge date date birth sex service medicine allergies patient recorded known allergies drugs attending chief complaint alcohol intoxication diabetic ketoacidosis major surgical invasive procedure none history present illness year old male poorly controlled type diabetes presents found sleeping sidewalk critically high glucose evidence alcohol intoxication ed glucose anion gap urine ketones head ct negative acute bleed started insulin drip admitted icu management time admission mildly responsive limiting history note prior admissions dka past medical history dm insulin dependent p mva social history construction worker illegal resident us denies smoking etoh illicit drugs family family history dm cad cancer physical exam physical exam admission vitals temp bp pulse resp l nc gen nad lying bed asleep arousable heent perrla anicteric sclera dry mucous membranes neck supple op clear lad cardiac rrr nl s1 s2 r g jvd pulmonary distant breath sounds wheezes rhonci rales abd

In [27]:
dr = df1.tail(1)['TEXT'].tolist()
dr[0]

'pm chest pa lat clip reason pneumonia admitting diagnosis diabetic ketoacidosis medical condition year old man fever reason examination pneumonia final report two view chest x ray indication fever comparison heart size normal recently reported right basilar opacity nearly resolved right pleural effusion resolved new areas abnormality impression rapidly resolving right basilar opacity time course favors atelectasis aspiration infectious pneumonia chest portable ap clip reason r consolidations effusions admitting diagnosis diabetic ketoacidosis medical condition year old man found dka etoh intoxication reports cough subjective fevers prior presentation r pna reason examination r consolidations effusions final report study ap chest history year old man dka alcohol intoxication patient cough subjective fevers findings comparison made previous study cardiac silhouette mediastinum unremarkable small right sided pleural effusion right basilar subsegmental atelectasis evidence pulmonary edema

In [11]:
l = df.sum(axis=0).tolist()

In [18]:
l[2:]

[72, 184, 105, 110, 205, 160, 240, 60, 78, 144]

In [13]:
x = l[2:]

In [15]:
x = [1-(i/813) for i in x]

In [16]:
x

[0.9114391143911439,
 0.7736777367773677,
 0.8708487084870848,
 0.8646986469864699,
 0.7478474784747847,
 0.8031980319803198,
 0.7047970479704797,
 0.9261992619926199,
 0.9040590405904059,
 0.8228782287822878]

In [2]:
df_pheno = pd.read_csv("/home/akshara/coding/Phenotyping/dump_files/pheno_disch_rad.csv")
df_icd = pd.read_csv("/home/akshara/coding/ICD/dump_files/disch-rad-icd.csv")


In [5]:
df_pheno

,HADM_ID,TEXT_x,TEXT_y,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEPRESSION,OBESITY,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS
0,100103.0,pm chest pa lat clip reason f u admitting diag...,admission date discharge date date birth sex f...,0,0,0,0,0,0,0,0,0,0
1,100473.0,chest portable ap clip reason please assess in...,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,1,0,0
2,100548.0,pm hip unilat min views left clip reason pleas...,admission date discharge date date birth sex s...,0,0,0,0,1,0,0,0,0,0
3,100654.0,chest portable ap clip reason eval pna effusio...,admission date discharge date date birth sex s...,0,0,0,1,0,0,0,0,0,0
4,100960.0,pm chest pa lat clip reason infiltrate medical...,admission date discharge date date birth sex f...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,199365.0,pm ct head w contrast clip reason ms changes h...,admission date discharge date date birth sex s...,0,1,0,0,0,0,1,0,0,0
637,199576.0,pm chest portable ap clip reason r pneumothora...,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,0,0,0
638,199652.0,ercp biliary pancreas gi unit clip reason bile...,admission date discharge date date birth sex f...,1,0,0,0,0,0,1,0,0,0
639,199807.0,pm chest portable ap physician reason eval pla...,admission date discharge date date birth sex f...,0,1,1,0,0,0,0,0,0,0


In [13]:
df1 = df_pheno.merge(df_icd, on='HADM_ID')

In [15]:
df1.columns

Index(['HADM_ID', 'TEXT_x_x', 'TEXT_y_x', 'ADVANCED.CANCER',
       'ADVANCED.HEART.DISEASE', 'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'CHRONIC.PAIN.FIBROMYALGIA',
       'DEPRESSION', 'OBESITY', 'OTHER.SUBSTANCE.ABUSE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS', 'G1', 'G2', 'G3', 'G4',
       'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15',
       'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [14]:
df1=df1.drop(['TEXT_x_y', 'TEXT_y_y'],axis=1)

In [10]:
df1

,HADM_ID,TEXT_x_x,TEXT_y_x,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEPRESSION,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,100103.0,pm chest pa lat clip reason f u admitting diag...,admission date discharge date date birth sex f...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,100473.0,chest portable ap clip reason please assess in...,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,100548.0,pm hip unilat min views left clip reason pleas...,admission date discharge date date birth sex s...,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,1,1
3,100654.0,chest portable ap clip reason eval pna effusio...,admission date discharge date date birth sex s...,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,100960.0,pm chest pa lat clip reason infiltrate medical...,admission date discharge date date birth sex f...,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,199365.0,pm ct head w contrast clip reason ms changes h...,admission date discharge date date birth sex s...,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
637,199576.0,pm chest portable ap clip reason r pneumothora...,admission date discharge date date birth sex s...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
638,199652.0,ercp biliary pancreas gi unit clip reason bile...,admission date discharge date date birth sex f...,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
639,199807.0,pm chest portable ap physician reason eval pla...,admission date discharge date date birth sex f...,0,1,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
